In [9]:
import json
import random
import sys
import numpy as np


#### Define the quadratic and cross-entropy cost functions

class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.

        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)


class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
#         Return the cost associated with an output ``a`` and desired output
#         ``y``.  Note that np.nan_to_num is used to ensure numerical
#         stability. 
        
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
#         Return the error delta from the output layer.
        
        return (a-y)


#### Main Network class
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
#         The list ``sizes`` contains the number of neurons in the respective
#         layers of the network.  For example, if the list was [2, 3, 1]
#         then it would be a three-layer network, with the first layer
#         containing 2 neurons, the second layer 3 neurons, and the
#         third layer 1 neuron. 

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
#         Initialize each weight using a Gaussian distribution with mean 0
#         and standard deviation 1 over the square root of the number of
#         weights connecting to the same neuron.  Initialize the biases
#         using a Gaussian distribution with mean 0 and standard
#         deviation 1.
        
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
#         Return the output of the network if ``a`` is input.
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0, #regularization parameter
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.

        """
        if evaluation_data:
                        
            evaluation_data = list(evaluation_data)
            n_data = len(evaluation_data)
            
        training_data = list(training_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print("Epoch {} finished".format(j))
           
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)                 
                print ("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print ("Accuracy on training data: {} / {}".format(
                    accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print ("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print ("Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data))
            print
        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
#         Update the network's weights and biases by applying gradient
#         descent using backpropagation to a single mini batch.
#         The mini_batch is a list of tuples (x, y)
#         eta = learning rate
#         lmbda = regularization parameter        
#         n = total size of the training data set.

        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
#         Returns a tuple (nabla_b, nabla_w) representing the gradient for the cost function C_x. 
#         nabla_b and nabla_w are layer-by-layer lists of numpy arrays

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
#         Return the total cost for the data set data.The flag
#         convert should be set to False if the data set is the
#         training data (the usual case), and to True if the data set is
#         the validation or test data.  See comments on the similar (but
#         reversed) convention for the accuracy method, above.
        
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost

    def save(self, filename):
        #Save the neural network to the file ``filename``.
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()


def load(filename):
#     Load a neural network from the file ``filename``.  Returns an
#     instance of Network.

    
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net


def vectorized_result(j):
#     Return a 10-dimensional unit vector with a 1.0 in the j'th position
#     and zeroes elsewhere.  This is used to convert a digit (0...9)
#     into a corresponding desired output from the neural network.

    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def sigmoid(z):
    #The sigmoid function.
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    #Derivative of the sigmoid function
    return sigmoid(z)*(1-sigmoid(z))

In [10]:
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip
import numpy as np

# Third-party libraries


def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('./mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='iso-8859-1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [12]:
training_data, validation_data, test_data = load_data_wrapper()

In [37]:
net = Network([784, 30, 10], cost=CrossEntropyCost)

In [38]:
net.SGD(training_data, 30, 10, 0.5,lmbda = 5.0,evaluation_data=validation_data,
 monitor_evaluation_accuracy=True,
 monitor_evaluation_cost=True,
 monitor_training_accuracy=True,
 monitor_training_cost=True)

Epoch 0 finished
Cost on training data: 0.49861334110702016
Accuracy on training data: 46918 / 50000
Cost on evaluation data: 0.8060758939928532
Accuracy on evaluation data: 9404 / 10000
Epoch 1 finished
Cost on training data: 0.44704183452272317
Accuracy on training data: 47613 / 50000
Cost on evaluation data: 0.8541352821096009
Accuracy on evaluation data: 9494 / 10000
Epoch 2 finished
Cost on training data: 0.4222718170221894
Accuracy on training data: 47898 / 50000
Cost on evaluation data: 0.8902788838316347
Accuracy on evaluation data: 9527 / 10000
Epoch 3 finished
Cost on training data: 0.40518845628272937
Accuracy on training data: 47967 / 50000
Cost on evaluation data: 0.8958818274455034
Accuracy on evaluation data: 9561 / 10000
Epoch 4 finished
Cost on training data: 0.3959622088613118
Accuracy on training data: 48121 / 50000
Cost on evaluation data: 0.9080913552600751
Accuracy on evaluation data: 9567 / 10000
Epoch 5 finished
Cost on training data: 0.3857796520157049
Accuracy

([0.8060758939928532,
  0.8541352821096009,
  0.8902788838316347,
  0.8958818274455034,
  0.9080913552600751,
  0.9146304304273882,
  0.9357288455719278,
  0.9216804361350406,
  0.9354999911181375,
  0.9544562972892887,
  0.9434817734097319,
  0.9273468769595854,
  0.9337514447400954,
  0.9932563665210454,
  0.9673846439002207,
  0.9398237025245462,
  0.9479745320717807,
  0.9757574843066421,
  0.9726472133398734,
  0.9412198988366378,
  0.9754419391969944,
  0.9401698918079984,
  0.9427986859370483,
  0.9269646148987378,
  0.9542678212924507,
  0.9517752643698862,
  0.9589403480139034,
  0.9446278746980826,
  0.9598055417475202,
  0.957016171823357],
 [9404,
  9494,
  9527,
  9561,
  9567,
  9574,
  9567,
  9590,
  9578,
  9571,
  9597,
  9620,
  9609,
  9506,
  9564,
  9604,
  9594,
  9540,
  9544,
  9604,
  9565,
  9603,
  9629,
  9646,
  9614,
  9592,
  9592,
  9608,
  9598,
  9602],
 [0.49861334110702016,
  0.44704183452272317,
  0.4222718170221894,
  0.40518845628272937,
  0.3959

In [4]:
net = Network([784, 30, 10])

In [7]:
net.SGD(training_data, 30, 10, 0.1, lmbda=5.0,evaluation_data=validation_data, monitor_evaluation_accuracy=True)

Epoch 0 finished
Accuracy on evaluation data: 0 / 0
Epoch 1 finished
Accuracy on evaluation data: 0 / 0
Epoch 2 finished
Accuracy on evaluation data: 0 / 0
Epoch 3 finished
Accuracy on evaluation data: 0 / 0
Epoch 4 finished
Accuracy on evaluation data: 0 / 0
Epoch 5 finished
Accuracy on evaluation data: 0 / 0
Epoch 6 finished
Accuracy on evaluation data: 0 / 0
Epoch 7 finished
Accuracy on evaluation data: 0 / 0
Epoch 8 finished
Accuracy on evaluation data: 0 / 0
Epoch 9 finished
Accuracy on evaluation data: 0 / 0
Epoch 10 finished
Accuracy on evaluation data: 0 / 0
Epoch 11 finished
Accuracy on evaluation data: 0 / 0
Epoch 12 finished
Accuracy on evaluation data: 0 / 0
Epoch 13 finished
Accuracy on evaluation data: 0 / 0
Epoch 14 finished
Accuracy on evaluation data: 0 / 0
Epoch 15 finished
Accuracy on evaluation data: 0 / 0
Epoch 16 finished
Accuracy on evaluation data: 0 / 0
Epoch 17 finished
Accuracy on evaluation data: 0 / 0
Epoch 18 finished
Accuracy on evaluation data: 0 / 0
Epo

([],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [],
 [])

In [13]:
net2 = Network([784, 30, 30, 10])

In [14]:
net2.SGD(training_data, 30, 10, 0.1, lmbda=5.0,evaluation_data=validation_data, monitor_evaluation_accuracy=True)

Epoch 0 finished
Accuracy on evaluation data: 9176 / 10000
Epoch 1 finished
Accuracy on evaluation data: 9431 / 10000
Epoch 2 finished
Accuracy on evaluation data: 9522 / 10000
Epoch 3 finished
Accuracy on evaluation data: 9567 / 10000
Epoch 4 finished
Accuracy on evaluation data: 9571 / 10000
Epoch 5 finished
Accuracy on evaluation data: 9626 / 10000
Epoch 6 finished
Accuracy on evaluation data: 9635 / 10000
Epoch 7 finished
Accuracy on evaluation data: 9625 / 10000
Epoch 8 finished
Accuracy on evaluation data: 9662 / 10000
Epoch 9 finished
Accuracy on evaluation data: 9655 / 10000
Epoch 10 finished
Accuracy on evaluation data: 9657 / 10000
Epoch 11 finished
Accuracy on evaluation data: 9680 / 10000
Epoch 12 finished
Accuracy on evaluation data: 9689 / 10000
Epoch 13 finished
Accuracy on evaluation data: 9683 / 10000
Epoch 14 finished
Accuracy on evaluation data: 9698 / 10000
Epoch 15 finished
Accuracy on evaluation data: 9703 / 10000
Epoch 16 finished
Accuracy on evaluation data: 969

([],
 [9176,
  9431,
  9522,
  9567,
  9571,
  9626,
  9635,
  9625,
  9662,
  9655,
  9657,
  9680,
  9689,
  9683,
  9698,
  9703,
  9691,
  9704,
  9704,
  9697,
  9693,
  9697,
  9696,
  9689,
  9710,
  9688,
  9702,
  9703,
  9703,
  9710],
 [],
 [])